In [1]:
!pip install vllm
!export VLLM_USE_MODELSCOPE=True
!pip install modelscope

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75

In [2]:
import subprocess
import os

# 启动 vLLM 服务（后台运行）
cmd = [
    "vllm", "serve", "BAAI/bge-m3",
    "--port", "8000",
    "--host", "0.0.0.0",
    "--gpu-memory-utilization", "0.2",
    "--trust-remote-code",
    "--served-model-name", "bge-m3",
    "--dtype", "float16"
]

# 启动进程并脱离 Jupyter 控制
process = subprocess.Popen(cmd, stdout=open("vllm.log", "w"), stderr=subprocess.STDOUT)
print("vLLM server started with PID:", process.pid)

vLLM server started with PID: 95


In [3]:
# 用于等待程序启动
import time
time.sleep(180)

In [4]:
!tail -n 20 vllm.log

INFO 07-19 01:33:30 [launcher.py:37] Route: /detokenize, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /v1/models, Methods: GET
INFO 07-19 01:33:30 [launcher.py:37] Route: /version, Methods: GET
INFO 07-19 01:33:30 [launcher.py:37] Route: /v1/chat/completions, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /v1/completions, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /v1/embeddings, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /pooling, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /classify, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /score, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /v1/score, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /v1/audio/transcriptions, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /v1/audio/translations, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /rerank, Methods: POST
INFO 07-19 01:33:30 [launcher.py:37] Route: /v1/rerank, Met

In [5]:
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"bge-m3","object":"model","created":1752888903,"owned_by":"vllm","root":"BAAI/bge-m3","parent":null,"max_model_len":8192,"permission":[{"id":"modelperm-f1262077e77a4654b7d9daacb5f876e7","object":"model_permission","created":1752888903,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [6]:
from openai import OpenAI
client = OpenAI(
    api_key="EMPTY",
    base_url="http://localhost:8000/v1"
)
response = client.embeddings.create(model="bge-m3", input=["测试文本"])
print(response.data[0].embedding[:50])

[-0.06409305334091187, 0.010989241302013397, -0.014208781532943249, -0.004913056269288063, 0.00889560952782631, -0.005075894296169281, 0.014311136677861214, 0.015102064236998558, -0.011277697049081326, 0.0016539691714569926, 0.00795114878565073, -0.006606571841984987, -0.012673451565206051, -0.0029799360781908035, -0.006862460169941187, -0.018991567194461823, -0.010477464646100998, -0.03558243811130524, -0.007699912879616022, -0.04421750456094742, -0.04406862333416939, -0.010551904328167439, 0.02499331161379814, 0.03465193510055542, 0.005024716258049011, 0.022946204990148544, -0.03911834955215454, -0.027970921248197556, -0.00927246268838644, 0.03063216060400009, 0.03896946832537651, 0.004640883766114712, 0.025942424312233925, -0.05441581830382347, -0.016348939388990402, -0.01681419089436531, 0.015492876060307026, -0.027915092185139656, -0.04161209613084793, -0.0012933992547914386, 0.0044943299144506454, -0.04507356882095337, -0.0013410875108093023, -0.03433556109666824, -0.001050305319